![LogoUC3M](https://upload.wikimedia.org/wikipedia/commons/4/47/Acronimo_y_nombre_uc3m.png)

Aprendizaje Automático · Grado en Ingeniería Informática · Curso 2022/23
---
**Cuaderno** - Práctica 2: Predicción del abandono (burnout) de empleados
<br><br>
***Grupo Reducido 82 - Grupo de laboratorio 13***

# **Carga de datos**
Para comenzar, importaremos las bibliotecas que necesitamos:

+ `biblioteca`: Definición.

 ################################### <br>
**(AQUÍ VAN LAS BIBLIOTECAS NECESARIAS)<br>
(AQUÍ VAN LAS BIBLIOTECAS NECESARIAS)<br>
(AQUÍ VAN LAS BIBLIOTECAS NECESARIAS)<br>
(AQUÍ VAN LAS BIBLIOTECAS NECESARIAS)<br>
(AQUÍ VAN LAS BIBLIOTECAS NECESARIAS)**<br>
 ################################### <br>

Una vez instaladas las bibliotecas necesarias, importamos **Numpy** y **Pandas** ya que los necesitaremos para el desarrollo de la práctica.
Destacar que nos referiremos a **Numpy** como *np* y a **Pandas** como *pd* por simplicidad.

In [1]:
import numpy as np
import pandas as pd

A continuación, cargaremos los datos necesarios. Dado que somos el **grupo 13**, usaremos los dos conjuntos de datos correspondientes.</br></br>Los datos serán divididos en un conjunto de **train** para entrenar y ajustar hiperparámetros, y un conjunto de **test** en el que evaluaremos las distintas posibilidades que se probarán en la práctica.</br></br>

In [8]:
import pickle

# Datos disponibles.
with open('attrition_available_13.pkl', 'rb') as f:
    data = pickle.load(f)

df = pd.DataFrame(data)

print(df)

# Mostramos la información del conjunto de datos.
print(f"El conjunto de datos tiene {len(df)} instancias.")

           hrs  absences  JobInvolvement  PerformanceRating  \
1     7.315971      13.0             2.0                4.0   
6     6.450877      17.0             3.0                4.0   
13         NaN      14.0             NaN                3.0   
28         NaN       NaN             2.0                3.0   
30    9.662808      11.0             NaN                3.0   
...        ...       ...             ...                ...   
4405  8.316921       6.0             3.0                3.0   
4406  5.897197       8.0             2.0                3.0   
4407  7.149526      18.0             3.0                NaN   
4408  9.187612       NaN             NaN                3.0   
4409  6.511790      17.0             NaN                3.0   

      EnvironmentSatisfaction  JobSatisfaction  WorkLifeBalance   Age  \
1                         3.0              2.0              NaN   NaN   
6                         1.0              3.0              1.0  28.0   
13                      

# **Análisis Exploratorio de Datos (EDA)**
Antes de comenzar, destacar que dado que los resultados deben ser reproducibles, hemos fijado la **semilla de números aleatorios** en los lugares adecuados. Para ello, hemos seleccionado como semilla el número del grupo de prácticas (*13*).</br></br>
Para poder realizar el análisis exploratorio de datos, llevaremos a cabo los siguientes pasos:

+ Número de instancias y atributos.
+ Tipo de atributos (numéricos o categóricos).
+ Columnas constantes o innecesarias.
+ Proporción de *missing values*.
+ Tipo de problema (clasificación o regresión).
+ Desbalance.

## **Número de instancias y atributos**

## **Tipo de atributos**

## **Columnas constantes o innecesarias**

## **Proporción de missing values**

## **Tipo de problema**

## **Desbalance**

## **Análisis de atributos**

# **Construcción de modelos**

 ################################### <br>
**(ENUNCIADO - BORRAR LUEGO)<br>
(ENUNCIADO - BORRAR LUEGO)<br>
(ENUNCIADO - BORRAR LUEGO)<br>
(ENUNCIADO - BORRAR LUEGO)<br>
(ENUNCIADO - BORRAR LUEGO)**<br>
 ################################### <br>

**Construcción de modelos**: para esta práctica usaremos **LogisticRegression** como método base (sin ajustar hiper-parámetros) y Boosting como método avanzado (ajustando hiper-parámetros), a elegir. Es importante realizar los preprocesos que los datos necesiten, usando preferentemente **pipelines**. Como método de boosting, se puede elegir uno de entre los métodos de **boosting** disponibles en scikit-learn.
<br><br>
**EXTRA**: Si además se usa uno de entre las **librerías externas** xgboost, lightgbm o catboost, se pueden sacar +0.35 puntos adicionales.

# **Mejora de resultados**

 ################################### <br>
**(ENUNCIADO - BORRAR LUEGO)<br>
(ENUNCIADO - BORRAR LUEGO)<br>
(ENUNCIADO - BORRAR LUEGO)<br>
(ENUNCIADO - BORRAR LUEGO)<br>
(ENUNCIADO - BORRAR LUEGO)**<br>
 ################################### <br>

Usando algún **método de selección de atributos de tipo filter (SelectKBest)** de entre los disponibles en sklearn **(f_classif, mutual_info_classif o chi2)**, comprobad si se pueden mejorar los resultados del apartado anterior y extraer conclusiones sobre qué atributos son más importantes, al menos de acuerdo a estos métodos.